# Python Project

**Hotel**

The program must present and display a report of the result obtained, showing for
each strategy the number of customers accommodated, the number of rooms occupied, the
number of different hotels occupied, the total volume of business (total earnings of each
hotel), and the degree of customer satisfaction (calculated according to the location of the
hotel assigned to them with respect to their preferences).
I will use 4 differents strategies:
**1-random**
**2-customer preference**
**3-price**
**4-availability**

**Random:**
customers are randomly distributed to the rooms until the seats or customers
are exhausted.


In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
hotels = pd.read_excel("/Users/raulspano/Desktop/progetto hotel/hotels.xlsx")

In [4]:
ospiti = pd.read_excel('/Users/raulspano/Desktop/progetto hotel/guests.xlsx')

In [5]:
preferenze = pd.read_excel('/Users/raulspano/Desktop/progetto hotel/preferences.xlsx')

In [6]:
hotels['stanze_disponibili'] = hotels['rooms'].copy()

In [7]:
guadagni_hotel={hotel: 0 for hotel in hotels['hotel']}

In [8]:
ospiti_allocati=0
stanze_occupate=0
hotel_occupati=set()
ospiti_soddisfatti=0

In [9]:
allocazioni=[]

In [10]:
print(allocazioni)

[]


In [11]:
for _, guest_row in ospiti.iterrows():
    guest=guest_row['guest']
    discount=guest_row['discount']
    ##dobbiamo prendere in considerazione le preferenze dell'ospite
    preferenze_ospite=preferenze[preferenze['guest']== guest]
    ##dobbiamo trovare stanze disponibili nelle preferenze del cliente
    hotels_preferiti=preferenze_ospite[preferenze_ospite['hotel'].isin(hotels[hotels['stanze_disponibili']>0]['hotel'])]
    if not hotels_preferiti.empty:
        ## se ci sono preferenze disponibili selezionarne una casualmente
        hotel_selezionato= np.random.choice(hotels_preferiti['hotel'])
        ospiti_soddisfatti += 1 ##vuol dire che l'ospite ha ottenuto un hotel preferito
    else:
        ## se non ci sono preferenze valide seleziona un hotel a caso tra quelli con stanze disponibili
        hotels_disponibili=hotels[hotels['stanze_disponibili']>0]
        if hotels_disponibili.empty:
            print('Non ci sono hotel disponibili')
            continue ##se non ci sono hotel disponibili passa al prossimo ospite
        ##selezioniamo casualmente un hotel disponibile
        hotel_selezionato= np.random.choice(hotels_disponibili['hotel'])
    
    ##recuperare il prezzo dell'hotel selezionato
    prezzo_hotel= hotels[hotels['hotel']==hotel_selezionato].iloc[0]
    price=prezzo_hotel['price']
    
    #calcoliamo lo sconto
    prezzo_finale= price*(1-discount)
    
    #salviamo le allocazioni
    allocazioni.append({
        'cliente':guest,
        'hotel_f': hotel_selezionato,
        'prezzo_pagato': prezzo_finale
    })
    
    ##dobbiamo ovviamente ridurre il numero di stanze disponibili nell'hotel
    indice_hotel=hotels[hotels['hotel']==hotel_selezionato].index
    hotels.loc[indice_hotel, 'stanze_disponibili'] -= 1
    
    ##dobbiamo aggiornare le statistiche
    ospiti_allocati += 1
    stanze_occupate += 1
    hotel_occupati.add(hotel_selezionato)
    guadagni_hotel[hotel_selezionato] += prezzo_finale

In [12]:
allocazioni_df= pd.DataFrame(allocazioni)  

In [13]:
numero_hotel_occupati= len(hotel_occupati)

In [14]:
print(f'Numero di ospiti che hanno ottenuto una camera: {ospiti_allocati}')
print(f'Numero di stanze occupate: {stanze_occupate}')
print(f'Numero di hotel occupati: {numero_hotel_occupati}')
print(f'Ospiti soddisfatti: {ospiti_soddisfatti}')


Numero di ospiti che hanno ottenuto una camera: 4000
Numero di stanze occupate: 4000
Numero di hotel occupati: 400
Ospiti soddisfatti: 3972


In [15]:
guadagni_df= pd.DataFrame(list(guadagni_hotel.items()), columns=['Hotel', 'Guadagno Totale'])
print('\nGuadagni totali di ogni hotel:')
print(guadagni_df)


Guadagni totali di ogni hotel:
         Hotel  Guadagno Totale
0      hotel_1          3207.75
1      hotel_2           911.72
2      hotel_3          1122.36
3      hotel_4          2103.80
4      hotel_5          1898.26
..         ...              ...
395  hotel_396           943.40
396  hotel_397           735.76
397  hotel_398          1288.26
398  hotel_399          3467.64
399  hotel_400          1599.35

[400 rows x 2 columns]
